<a href="https://colab.research.google.com/github/segmue/GIR_Project/blob/main/Irchel_Geoparser_ACLED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install geoparser==0.1.8
!python -m spacy download en_core_web_sm
!python -m geoparser download geonames

In [2]:
# prompt: Mount google drive and load all csvs in the folder GEO876/data/ into panda dataframes

from google.colab import drive
import pandas as pd
import os
import pickle

drive.mount('/content/drive')

# Path to the folder containing the CSV files
data_folder = '/content/drive/MyDrive/GEO871'  # Replace with the actual path

def load_files(data_folder):
# List to store dataframes
  dataframes = {}
  # Iterate through files in the directory
  for filename in os.listdir(data_folder):
      if filename.endswith(".csv"):
          filepath = os.path.join(data_folder, filename)
          try:
              # Read the CSV file into a pandas DataFrame
              df = pd.read_csv(filepath)
              dataframes[filename] = df
              print(f"Successfully loaded '{filename}' into a DataFrame.")
          except Exception as e:
              print(f"Error loading '{filename}': {e}")
  return dataframes


def split_batches(df, batchsize):
  batches = {}
  num_batches = (len(df) + batchsize - 1) // batchsize  # Calculate the number of batches
  for i in range(num_batches):
      start_index = i * batchsize
      end_index = min((i + 1) * batchsize, len(df))  # Handle the last batch
      batches[i] = df.iloc[start_index:end_index]
  return batches

def write_df_to_drive(df, filename):
  """Writes a DataFrame to a pickle file in the Google Drive data folder.

  Args:
    df: The DataFrame to write.
    filename: The name of the pickle file (including the .pkl extension).
  """
  filepath = os.path.join(data_folder, filename)
  try:
    with open(filepath, 'wb') as f:
      pickle.dump(df, f)
    print(f"DataFrame successfully written to '{filepath}'")
  except Exception as e:
    print(f"Error writing DataFrame to '{filepath}': {e}")



from geoparser import Geoparser
geo = Geoparser(spacy_model='en_core_web_sm', transformer_model='dguzh/geo-all-MiniLM-L6-v2', gazetteer='geonames')

def geoparse_df_notes(df):
  notes_list = df["notes"].tolist()
  parsed_test = geo.parse(notes_list)

  toponyms = []
  locations = []
  res = []
  for row in parsed_test:
    entry = [str(a) for a in row.toponyms]
    toponyms.append(entry)
    locdata = row.locations.data
    locations.append(locdata)
    res = {}
    for i, toponym in enumerate(entry):
      if locs[i] is not None:
        subres = {toponym: {"latitude": locs[i].latitude, "longitude": locs[i].longitude}}
      else:
        subres = {toponym: {"latitude": None, "longitude": None}}
      res.update(subres)
    res.append(res)


  df["geoparser_locations"] = pd.Series(locations)
  df["geoparser_toponyms"] = pd.Series(toponyms)
  df["geoparser_res"] = pd.Series(res)
  return df

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.40k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [117]:
loc = df.iloc[0]["geoparser_locations"]
for a in loc.data:
  print(a)

{'geonameid': 703611, 'name': 'Kurilovka', 'admin2_geonameid': 706375, 'admin2_name': 'Khmelnytskyy Raion', 'admin1_geonameid': 706370, 'admin1_name': 'Khmelnytskyi', 'country_geonameid': 690791, 'country_name': 'Ukraine', 'feature_name': 'populated place', 'latitude': 49.58631, 'longitude': 26.31226, 'elevation': None, 'population': 0}
{'geonameid': 701750, 'name': 'Martynivka', 'admin2_geonameid': 706375, 'admin2_name': 'Khmelnytskyy Raion', 'admin1_geonameid': 706370, 'admin1_name': 'Khmelnytskyi', 'country_geonameid': 690791, 'country_name': 'Ukraine', 'feature_name': 'populated place', 'latitude': 49.48002, 'longitude': 26.77238, 'elevation': None, 'population': 0}
{'geonameid': 6641174, 'name': 'Melovoy', 'admin2_geonameid': None, 'admin2_name': None, 'admin1_geonameid': 479119, 'admin1_name': 'Ulyanovsk', 'country_geonameid': 2017370, 'country_name': 'Russia', 'feature_name': 'populated place', 'latitude': 54.11816, 'longitude': 48.32531, 'elevation': None, 'population': 0}
{'ge

In [89]:
# prompt: Implement function load_df_from_drive, which takes a file pattern (e.g. Europe-Central-Asia_2018-2024_Nov22_) and loads ALL files with .pkl ending, which mach the pattern in a dataframe (saved in my google drive data_folder = '/content/drive/MyDrive/GEO871'  # Replace with the actual path)

import pandas as pd
import os
import re

def load_df_from_drive(file_pattern):
    """Loads all .pkl files matching a pattern from Google Drive into a DataFrame.

    Args:
        file_pattern: The file pattern to match (e.g., "Europe-Central-Asia_2018-2024_Nov22_").

    Returns:
        A pandas DataFrame containing the concatenated data from all matching files, or None if no files match or an error occurs.
    """
    all_dfs = []
    for filename in os.listdir(data_folder):
      if filename.endswith("final.pkl"):
        continue

      if filename.endswith(".pkl") and re.search(file_pattern, filename):
            filepath = os.path.join(data_folder, filename)
            try:
                with open(filepath, 'rb') as f:
                    df = pickle.load(f)
                    all_dfs.append(df)
                    print(f"Loaded: {filename}")
            except (FileNotFoundError, pickle.UnpicklingError) as e:
                print(f"Error loading {filename}: {e}")
                return None

    if not all_dfs:
        print(f"No files found matching the pattern '{file_pattern}'")
        return None

    combined_df = pd.concat(all_dfs, ignore_index=True)
    return combined_df


import pandas as pd

def generate_geoparser_res(row):
    geoparser_locations = row.get("geoparser_locations", None)
    geoparser_toponyms = row.get("geoparser_toponyms", None)

    # Initialize result dictionary
    res = {}

    # Handle cases where geoparser_toponyms is not valid
    if not isinstance(geoparser_toponyms, (list, tuple)):
        return res

    # Process each toponym and location data
    for i, toponym in enumerate(geoparser_toponyms):
      location_data = geoparser_locations.data[i]

        # Add latitude and longitude if valid
      if location_data and "latitude" in location_data and "longitude" in location_data:
            subres = {toponym: {
                "latitude": location_data["latitude"],
                "longitude": location_data["longitude"]
            }}
            res.update(subres)

    return res



In [90]:
df = load_df_from_drive("Europe-Central-Asia_2018-2024_Nov22_")


Loaded: Europe-Central-Asia_2018-2024_Nov22_0.pkl
Loaded: Europe-Central-Asia_2018-2024_Nov22_1.pkl
Loaded: Europe-Central-Asia_2018-2024_Nov22_2.pkl
Loaded: Europe-Central-Asia_2018-2024_Nov22_3.pkl
Loaded: Europe-Central-Asia_2018-2024_Nov22_4.pkl
Loaded: Europe-Central-Asia_2018-2024_Nov22_5.pkl
Loaded: Europe-Central-Asia_2018-2024_Nov22_6.pkl
Loaded: Europe-Central-Asia_2018-2024_Nov22_7.pkl
Loaded: Europe-Central-Asia_2018-2024_Nov22_8.pkl
Loaded: Europe-Central-Asia_2018-2024_Nov22_9.pkl
Loaded: Europe-Central-Asia_2018-2024_Nov22_10.pkl
Loaded: Europe-Central-Asia_2018-2024_Nov22_11.pkl
Loaded: Europe-Central-Asia_2018-2024_Nov22_12.pkl
Loaded: Europe-Central-Asia_2018-2024_Nov22_13.pkl
Loaded: Europe-Central-Asia_2018-2024_Nov22_14.pkl
Loaded: Europe-Central-Asia_2018-2024_Nov22_15.pkl
Loaded: Europe-Central-Asia_2018-2024_Nov22_16.pkl
Loaded: Europe-Central-Asia_2018-2024_Nov22_17.pkl
Loaded: Europe-Central-Asia_2018-2024_Nov22_18.pkl
Loaded: Europe-Central-Asia_2018-2024_Nov

In [91]:
len(df)

425663

In [92]:
len(df[df["geoparser_toponyms"].str.len() == 0])

2820

In [93]:
df["geoparser_res"] = df.apply(generate_geoparser_res, axis=1)


In [108]:
row = df.iloc[1][["geoparser_res", "geoparser_toponyms", "geoparser_locations"]]
specific_types = row.apply(type)
print(specific_types)

geoparser_res                                       <class 'dict'>
geoparser_toponyms                                  <class 'list'>
geoparser_locations    <class 'geoparser.geodoc.GeoDoc.Locations'>
Name: 1, dtype: object


In [100]:
df[df["geoparser_res"] == {}][["geoparser_res","geoparser_toponyms","geoparser_locations"]]

,geoparser_res,geoparser_toponyms,geoparser_locations
2,{},[],[]
3,{},[],[]
4,{},[],[]
6,{},[Havendoklaan],[None]
7,{},[the Burgemeester Reynaertstraat],[None]
...,...,...,...
425658,{},NaN,NaN
425659,{},NaN,NaN
425660,{},NaN,NaN
425661,{},NaN,NaN


In [94]:
len(df[df["geoparser_res"].str.len() == 0])

418870

In [87]:
empty_res = df[df["geoparser_res"].str.len() == 0]
filtered_res = empty_res[~empty_res["geoparser_locations"].isna()]
len(filtered_res)

def is_empty_location(value):
    # Check if value is None, "None", empty list, or list containing only None
    return value in [None, "None", []] or (isinstance(value, list) and all(v is None for v in value))

# Apply filter
filtered_res = empty_res[~empty_res["geoparser_locations"].apply(is_empty_location)]
len(filtered_res)

418870

In [88]:
filtered_res

,event_id_cnty,event_date,year,time_precision,disorder_type,event_type,sub_event_type,actor1,assoc_actor_1,inter1,...,geo_precision,source,source_scale,notes,fatalities,tags,timestamp,geoparser_locations,geoparser_toponyms,geoparser_res
2,RUS27077,2024-11-22,2024,1,Political violence,Battles,Armed clash,Military Forces of Russia (2000-),NaN,State forces,...,2,Ministry of Defence of Russia,Regional,"On 22 November 2024, Russian forces, engaged i...",0,NaN,1732644602,[],[],{}
3,UKR194942,2024-11-22,2024,1,Political violence,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Russia (2000-),NaN,External/Other forces,...,1,Ministry of Defence of Ukraine,Other,"On 22 November 2024, Russian forces attacked I...",0,NaN,1732644609,[],[],{}
4,UKR195013,2024-11-22,2024,1,Political violence,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Russia (2000-),NaN,External/Other forces,...,1,Ministry of Defence of Ukraine,Other,"On 22 November 2024, Russian forces attacked R...",0,NaN,1732644610,[],[],{}
6,BEL3692,2024-11-22,2024,1,Demonstrations,Protests,Peaceful protest,Protesters (Belgium),Labor Group (Belgium),Protesters,...,1,Het Nieuwsblad,Subnational,"On 22 November 2024, workers at Van Heck Inter...",0,crowd size=no report,1732654481,[None],[Havendoklaan],{}
7,BEL3695,2024-11-22,2024,1,Demonstrations,Protests,Peaceful protest,Protesters (Belgium),PVDA-PTB: Workers Party of Belgium; Women (Bel...,Protesters,...,1,Het Nieuwsblad,Subnational,"On 22 November 2024, Zelle, about 30 protestor...",0,crowd size=about 30,1732654481,[None],[the Burgemeester Reynaertstraat],{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425658,AZE6887,2018-01-01,2018,1,Political violence,Battles,Armed clash,Military Forces of Artsakh (2007-),NaN,State forces,...,2,Ministry of Defence of Azerbaijan,Other,"On 1 January 2018, military forces of Artsakh ...",0,NaN,1728339603,NaN,NaN,{}
425659,UKR12,2018-01-01,2018,1,Demonstrations,Protests,Peaceful protest,Protesters (Ukraine),National Corps Party; OUN: Organization of Ukr...,Protesters,...,1,EuroNews; Ukraine General Newswire,National-Regional,"On 1 January 2018, people, including members o...",0,crowd size=no report,1732648474,NaN,NaN,{}
425660,UKR3,2018-01-01,2018,1,Demonstrations,Protests,Peaceful protest,Protesters (Ukraine),National Corps Party; OUN: Organization of Ukr...,Protesters,...,1,EuroNews; Ukraine General Newswire,National-Regional,"On 1 January 2018, people, including members o...",0,crowd size=no report,1732648589,NaN,NaN,{}
425661,UKR39146,2018-01-01,2018,1,Demonstrations,Protests,Peaceful protest,Protesters (Ukraine),Svoboda; National Corps Party; Sokil; Right Se...,Protesters,...,1,Svoboda,National,"On 1 January 2018, people, including members o...",0,crowd size=no report,1732648590,NaN,NaN,{}


In [ ]:
df = load_df_from_drive("Europe-Central-Asia_2018-2024_Nov22_")
df["geoparser_res"] = df.apply(generate_geoparser_res, axis=1)
df = df.drop(columns=["geoparser_locations", "geoparser_toponyms"])
write_df_to_drive(df, "Europe-Central-Asia_2018-2024_Nov22_final.pkl")


df = load_df_from_drive("Europe-Central-Asia_2018-2024_Nov22_")
df["geoparser_res"] = df.apply(generate_geoparser_res, axis=1)
df = df.drop(columns=["geoparser_locations", "geoparser_toponyms"])
write_df_to_drive(df, "Europe-Central-Asia_2018-2024_Nov22_final.pkl")

DataFrame successfully written to '/content/drive/MyDrive/GEO871/Europe-Central-Asia_2018-2024_Nov22_final.pkl'


In [ ]:
df = df.drop(columns=["geoparser_locations", "geoparser_toponyms"])

In [ ]:
df.iloc[8]["geoparser_res"]

{'Ukraine': {'latitude': 49.0, 'longitude': 32.0}}

In [ ]:
df.iloc[0]["notes"]

"On 22 November 2024, Russian military carried out air strikes and shelled with artillery concentration of Ukrainian military personnel and equipment in the area around Nizhnii Klin (Sudzhanskiy, Kursk). According to the Russian MoD, on 22 November in the results of air strikes and shelling Ukrainian military's losses amounted to more than 150 servicemen, including near Kurilovka, Martynovka, Melovoy, Nizhnii Klin, Nikolayevka, Nikolayevo-Daryino, Plekhovo, Staraya Sorochina, and Cherkasskoe Porechnoe. [Russian MoD reported 150 fatalities. Coded as 10 fatalities split across 9 events. 1 fatality coded to this event]."

In [ ]:
oneloc = df.iloc[0]["geoparser_locations"]
one_toponym = df.iloc[0]["geoparser_toponyms"]

res = {}
for i in range(len(one_toponym)):
  toponym = one_toponym[i]
  location_data = oneloc.data[i]
  subres = {toponym: {"latitude": location_data["latitude"], "longitude": location_data["longitude"]}}
  res.update(subres)

In [ ]:
res

{'Kurilovka': {'latitude': 49.58631, 'longitude': 26.31226},
 'Martynovka': {'latitude': 49.48002, 'longitude': 26.77238},
 'Melovoy': {'latitude': 54.11816, 'longitude': 48.32531},
 'Plekhovo': {'latitude': 51.53652, 'longitude': 30.90738}}

In [ ]:
oneloc.data[0]

{'geonameid': 703611,
 'name': 'Kurilovka',
 'admin2_geonameid': 706375,
 'admin2_name': 'Khmelnytskyy Raion',
 'admin1_geonameid': 706370,
 'admin1_name': 'Khmelnytskyi',
 'country_geonameid': 690791,
 'country_name': 'Ukraine',
 'feature_name': 'populated place',
 'latitude': 49.58631,
 'longitude': 26.31226,
 'elevation': None,
 'population': 0}

In [ ]:
data = load_files(data_folder)


Successfully loaded 'Asia-Pacific_2018-2024_Dec06.csv' into a DataFrame.
Successfully loaded 'Europe-Central-Asia_2018-2024_Nov22.csv' into a DataFrame.


In [ ]:
for filename, df in data.items():
  batches = split_batches(df, 10000)
  data[filename] = batches

In [ ]:
for filename, df in data.items():
  for batch_num, batch_df in df.items():
    if filename == 'Europe-Central-Asia_2018-2024_Nov22.csv' and batch_num <= 4:
      print(f"Skipping '{filename}' batch {batch_num}")
      continue
    result = geoparse_df_notes(batch_df)
    write_filename = filename[:-4] + "_" + str(batch_num) + ".pkl"
    write_df_to_drive(result, write_filename)
    print(f"Successfully wrote '{write_filename}'")


Skipping 'Asia-Pacific_2018-2024_Dec06.csv' batch 0
Skipping 'Asia-Pacific_2018-2024_Dec06.csv' batch 1
Skipping 'Asia-Pacific_2018-2024_Dec06.csv' batch 2
Skipping 'Asia-Pacific_2018-2024_Dec06.csv' batch 3
Skipping 'Asia-Pacific_2018-2024_Dec06.csv' batch 4
Skipping 'Asia-Pacific_2018-2024_Dec06.csv' batch 5
Skipping 'Asia-Pacific_2018-2024_Dec06.csv' batch 6
Skipping 'Asia-Pacific_2018-2024_Dec06.csv' batch 7
Skipping 'Asia-Pacific_2018-2024_Dec06.csv' batch 8
Skipping 'Asia-Pacific_2018-2024_Dec06.csv' batch 9
Skipping 'Asia-Pacific_2018-2024_Dec06.csv' batch 10
Skipping 'Asia-Pacific_2018-2024_Dec06.csv' batch 11
Skipping 'Asia-Pacific_2018-2024_Dec06.csv' batch 12
Skipping 'Asia-Pacific_2018-2024_Dec06.csv' batch 13
Skipping 'Asia-Pacific_2018-2024_Dec06.csv' batch 14
Skipping 'Asia-Pacific_2018-2024_Dec06.csv' batch 15
Skipping 'Asia-Pacific_2018-2024_Dec06.csv' batch 16
Skipping 'Asia-Pacific_2018-2024_Dec06.csv' batch 17
Skipping 'Asia-Pacific_2018-2024_Dec06.csv' batch 18
Ski

Batches:   0%|          | 0/10000 [00:00<?, ?it/s]

Toponym Resolution...


Batches:   0%|          | 0/5352 [00:00<?, ?it/s]

Batches:   0%|          | 0/1236 [00:00<?, ?it/s]

...

Toponym Resolution...


Batches:   0%|          | 0/1254 [00:00<?, ?it/s]

Batches:   0%|          | 0/582 [00:00<?, ?it/s]

DataFrame successfully written to '/content/drive/MyDrive/GEO871/Europe-Central-Asia_2018-2024_Nov22_4.pkl'
Successfully wrote 'Europe-Central-Asia_2018-2024_Nov22_4.pkl'
Toponym Recognition...


Batches:   0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
strs = ["my first reference is Zurich, in Switzerland", "my second is katwa in Kongo"]

In [ ]:
parsed_text = geo.parse(strs)

In [ ]:
a = [str(a) for a in parsed_text.toponyms]
b = parsed_text.locations

In [ ]:
for r in parsed_text:
  print(r)